# Example 2 - Networked Controlled Metronome

The second example demonstrates how we can begin to control and synchronize audio over the network using OSC. The code expands on example 1, but here we continously send "tick" messages to the remote client to control a metronome. Similarly, we listen for "tick" messages on our server to control a local metronome audio playback.


### Files 
1. Notebook "Example 2 - Networked Controlled Metronome"
2. Metronome tick "./tick.wav"
3. Pure Data patch example1.pd from "../pd/example2.pd"
4. Metronome tick "../pd/tick.wav"

<p align="left">
 <img src="../../fig/example2.jpg" width=500>
</p>

In [ ]:
# setup our connection variables 
# remote client ip and port
clientIp = '127.0.0.1'
clientPort = 8001

# local server ip and port
serverIp = '127.0.0.1'
serverPort = 8000

## Connect with PD client

In [ ]:
from pythonosc import udp_client

# Setup a client connection and send messages to PD
client = udp_client.UDPClient(clientIp, clientPort)
print(f'Starting client on {clientIp}, port {clientPort}.')

In [ ]:
from pythonosc import osc_bundle_builder
from pythonosc import osc_message_builder
import time

# send OSC frames (custom bundles) with tick messages to the client
while True:
    # open a OSC bundle
    bundle = osc_bundle_builder.OscBundleBuilder(
        osc_bundle_builder.IMMEDIATELY)

    # create a message with an OSC address
    msg = osc_message_builder.OscMessageBuilder(address="/pd")

    # add arguments to the message
    msg.add_arg("tick")

    # build the message and add to current bundle
    bundle.add_content(msg.build())

    # close OSC bundle
    bundle = bundle.build()

    # send the bundle to remote client
    client.send(bundle)

    # One frame every half second
    time.sleep(.5)

print("done sending...")

## Serve an OSC server

In [ ]:
from pythonosc import dispatcher
from pythonosc import osc_server
from playsound import playsound

audio = "./tick.wav"

# Setup different "routes" where we can map different functions to different OSC adressess we receive.
def pyHandler(address, args):
    print(f'{address} {args}')
    
    if args == "tick":
        # play the tick audio file from the root dir
        playsound(audio)

dispatcher = dispatcher.Dispatcher()
dispatcher.map("/py*", pyHandler)

# A simple OSC threading server to listen for OSC messages
server = osc_server.ThreadingOSCUDPServer((serverIp, serverPort), dispatcher)
print(f'Starting server on {server.server_address[0]}, port {server.server_address[1]}.')

# run our server
server.serve_forever()

In [ ]:
# closing the OSC server
server.server_close()

## Activity

(same as example 1)

1. Test and explore the example on your local machine. Use "localhost" or "127.0.0.1" as your client and server IPs.
2. Test and play around with the example on a Wi-Fi or wired network in pairs.

Most likely, you will notice that controlling the metronome over Wi-Fi is quite unstable, with periodic dropouts due to network jitter and latency.